In [ ]:
import sys; sys.path.append("C:/Users/SpinAge_SHNO/OneDrive - Aarhus universitet/VS/SpinAge/Setup_control")
from shno_lib_docstring import *
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
import h5py

# Initilize the intruments
magnet = rs_hmp_psu()             # Default COM port
lna_psu = keysight36300()         # Default IP
shno_bias = keithley2450(addr=2)  # IP Connection
vxm = Velmex_VXM()                # Default COM port
fsv = rs_fsv()                    # Default IP  

## Initial control/setup cells

In [ ]:
# Turn off all power
lna_psu.off()
shno_bias.off()
magnet.off()


In [ ]:
# Control angle 
vxm.move_degree(-18-18)

In [ ]:
# Test SHNO connection
shno_bias.set_output(V=2, I=1e-3, turn_on=True)

## Start magnetic field sweep

In [ ]:
# initialize the parameteters

# Metadata
sample_name = "X6_L1R2"
date = time.localtime()[1:3]
date = f"{date[0]}-{date[1]}"

# Angle: 82 degrees
bias_current = 2.0e-3 # SHNO bias: 180nm: 2.2mA, 150nm: 2mA
shno_bias.set_output(I=bias_current, V=2, turn_on=False)

# Set magnet range in output current
start_field = 1     #[A]
stop_field = 5      #[A]
step_size = 0.05    #[A]

magnet_range = np.linspace(start_field, stop_field, int(abs((stop_field-start_field)/step_size)+1))
applied_field = np.zeros_like(magnet_range)

# Turn on FSV Spectrum Analyzer
start_f = 4E9; stop_f = 10E9
bandwidth = 500E3
capture_points = int(2*(stop_f-start_f)/bandwidth/2)

waveforms = np.zeros((len(magnet_range), capture_points))
waveforms_bg = np.zeros_like(waveforms)
fsv.show_screen(start_f = start_f, stop_f = stop_f, bandwidth=bandwidth, points = capture_points)
frequency = np.linspace(start_f, stop_f, capture_points)



In [ ]:
# Start measurements
lna_psu.lna(state=True)
shno_bias.on()
time.sleep(1)

progress_bar = tqdm(enumerate(magnet_range), total=len(magnet_range))
mag_R = magnet.set_calibrated_current(start_field)

# Run measurement loop
for i, field in progress_bar:
    mag_R = magnet.set_calibrated_current(field, mag_R)
    time.sleep(1)
    applied_field[i] = magnet.field_strenght()
    progress_bar.set_postfix({'Applied Field': applied_field[i]})
    waveforms[i] = fsv.get_trace(avg_count=50)

# Turn off all power
lna_psu.lna(state=False)
shno_bias.off()
magnet.off()
shno_bias.chirp(0)


In [ ]:
# Save the data
name = f'field_sweep/{date}_{sample_name}_{str(bias_current*1000).replace(".", "_")}_82deg.h5'
ans = input(f"Confirm saving? [y/n]:{name}")
if ans == 'y':
    with h5py.File(name, 'w') as f:
        # Store the waveforms, frequency, and applied field
        f.create_dataset('frequency', data=frequency)
        f.create_dataset('waveforms', data=waveforms)
        f.create_dataset('applied_field', data=applied_field)
    print(f"File saved as: '{name}'")
    print("Time:", time.strftime("%H:%M:%S", time.localtime()))
else: 
    print("Save cancelled!")

## Start Angle Sweep

In [ ]:
# Control the angle to set the starting angle for the measurement 
vxm.move_degree(-8)

In [ ]:
# Get postition relative to the position of the stage when the VXM was initialized
vxm.get_pos()

In [ ]:
# initialize the parameteters

# Metadata
sample_name = "X6_L1R2"
date = time.localtime()[1:3]
date = f"{date[0]}-{date[1]}"

# Angle: 82 degrees
bias_current = 2.0e-3 # SHNO bias: 180nm: 2.2mA, 150nm: 2mA
shno_bias.set_output(I=bias_current, V=2, turn_on=False)

# Set magnet current 
magnet_current = 3.3 #[A] # 3.3A = 0.5 T

# Angle
total_angle = 10  #[degrees]
step_size = 0.025 #[degrees]
angle_steps = int(total_angle/step_size)
angle_range = np.linspace(0, total_angle, angle_steps)
applied_field = np.zeros(angle_steps)

# Turn on FSV
start_f = 3E9; stop_f = 10E9
bandwidth = 500E3           # noise floor approx -80dBm @ 500e3
capture_points = int(2*(stop_f-start_f)/bandwidth)
waveforms = np.zeros((angle_steps, capture_points))
fsv.show_screen(start_f = start_f, stop_f = stop_f, bandwidth=bandwidth, points = capture_points)
frequency = np.linspace(start_f, stop_f, capture_points)



In [ ]:
# Start measurements
lna_psu.lna(state=True)
shno_bias.on()
time.sleep(1)
mag_R = magnet.set_calibrated_current(magnet_current)

# Run the measuement loop
for i in tqdm(range(angle_steps)):
    mag_R = magnet.set_calibrated_current(magnet_current, mag_R)
    applied_field[i] = magnet.field_strenght()
    waveforms[i] = fsv.get_trace(avg_count=50)
    vxm.Move_degree(step_size)
    shno_bias.on()
    time.sleep(0.5)

# Turn off all power
lna_psu.lna(state=False)
shno_bias.off()
magnet.off()
shno_bias.chirp(0)

In [ ]:
# Save the data
name = f'angle_sweep/{date}_{sample_name}_{str(bias_current*1000).replace(".", "_")}_start_72deg_field_{magnet_current}A.h5'
ans = input(f"Confirm saving? [y/n]:{name}")
if ans == 'y':
    with h5py.File(name, 'w') as f:
        # Store the waveforms, frequency, and applied field
        f.create_dataset('frequency', data=frequency)
        f.create_dataset('waveforms', data=waveforms)
        f.create_dataset('applied_field', data=applied_field)
        f.create_dataset('angle_range', data=angle_range)
    print(f"File saved as: '{name}'")
    print("Time:", time.strftime("%H:%M:%S", time.localtime()))
else: 
    print("Save cancelled!")

## Bias sweep

In [ ]:
# initialize the parameteters

# Metadata
sample_name = "X6_L2R4"
date = time.localtime()[1:3]
date = f"{date[0]}-{date[1]}"

# Angle: 82 degrees

# Set magnet range in output voltage
magnet_field = 3.3 #3.3A = 0.5 mT

# Set bias current range
start_bias = 5e-3
stop_bias = 20e-3
bias_step = 0.25e-3

total_steps = int((stop_bias-start_bias)/bias_step)
bias_range = np.linspace(start_bias, stop_bias, total_steps)
applied_field = np.zeros_like(bias_range)

# Turn on FSV
start_f = 3E9; stop_f = 10E9
bandwidth = 500E3
capture_points = int(2*(stop_f-start_f)/bandwidth)

waveforms = np.zeros((total_steps, capture_points))
fsv.show_screen(start_f = start_f, stop_f = stop_f, bandwidth=bandwidth, points = capture_points)
frequency = np.linspace(start_f, stop_f, capture_points)


In [ ]:
# Start measurements
lna_psu.lna(state=True)
mag_R = magnet.set_calibrated_current(magnet_field, 2)
time.sleep(1)

# Run the measuement loop
for i in tqdm(range(total_steps)):
    mag_R = magnet.set_calibrated_current(magnet_field, mag_R)
    shno_bias.set_output(2, bias_range[i], turn_on=True)
    time.sleep(1)
    applied_field[i] = magnet.field_strenght()
    progress_bar.set_postfix({'Applied Field': applied_field[i]})
    waveforms[i] = fsv.get_trace(avg_count=50)

# Turn off all power
lna_psu.lna(state=False)
shno_bias.off()
magnet.off()
shno_bias.chirp(0)

In [ ]:
# Save the data
name = f'bias_sweep/{date}_{sample_name}_76deg_field_{magnet_field}A.h5'
ans = input(f"Confirm saving? [y/n]:{name}")
if ans == 'y':
    with h5py.File(name, 'w') as f:
        # Store the waveforms, frequency, and applied field
        f.create_dataset('frequency', data=frequency)
        f.create_dataset('waveforms', data=waveforms)
        f.create_dataset('applied_field', data=applied_field)
        f.create_dataset('bias_range', data=bias_range)
    print(f"File saved as: '{name}'")
    print("Time:", time.strftime("%H:%M:%S", time.localtime()))
else: 
    print("Save cancelled!")

## Angle/Field sweep

In [ ]:
# Move stage to stating position
vxm.Move_degree(18)

In [ ]:
# initialize the parameteters

# Metadata
sample_name = "X6_L2R4"
date = time.localtime()[1:3]
date = f"{date[0]}-{date[1]}"

# The set bias current
bias_current = 2.0e-3 # SHNO bias: 180nm: 2.2mA, 150nm: 2mA
shno_bias.set_output(I=bias_current, V=2, turn_on=False)

# Set magnet range in current
start_field = 2.5
stop_field = 4.5
step_size = 0.05
magnet_range = np.linspace(start_field, stop_field, int(abs((stop_field-start_field)/step_size)+1))
applied_field = np.zeros_like(magnet_range)


# Angle
start_angle = 76 # set manually 
total_angle = 6
step_size = 0.5
angle_steps = int(total_angle/step_size)
angle_range = np.linspace(0, total_angle, angle_steps+1)

# Turn on FSV
start_f = 3E9; stop_f = 9E9
bandwidth =750E3
capture_points = int(2*(stop_f-start_f)/bandwidth)
waveforms = np.zeros((len(magnet_range), capture_points))
waveforms_bg = np.zeros_like(waveforms)
fsv.show_screen(start_f = start_f, stop_f = stop_f, bandwidth=bandwidth, points = capture_points)
frequency = np.linspace(start_f, stop_f, capture_points)


In [ ]:
# Start measurements
lna_psu.lna(state=True)
shno_bias.on()
time.sleep(1)

for angle in tqdm(angle_range):

    progress_bar = tqdm(enumerate(magnet_range), total=len(magnet_range), leave=False)
    mag_R = magnet.set_calibrated_current(start_field)
    for i, field in progress_bar:
        mag_R = magnet.set_calibrated_current(field, mag_R)
        applied_field[i] = magnet.field_strenght()
        progress_bar.set_postfix({'Applied Field': applied_field[i]})
        waveforms[i] = fsv.get_trace(avg_count=50)
        
    # Save the data for each field sweep automatically
    name = f'angle_sweep/field_sweep/{sample_name}/{date}_{str(bias_current*1000).replace(".", "_")}mA_{angle + start_angle}deg.h5'
    with h5py.File(name, 'w') as f:
        # Store the waveforms, frequency, and applied field
        f.create_dataset('frequency', data=frequency)
        f.create_dataset('waveforms', data=waveforms)
        f.create_dataset('applied_field', data=applied_field)

    vxm.Move_degree(step_size)
    
# Turn off all power
lna_psu.lna(state=False)
shno_bias.off()
magnet.off()
shno_bias.chirp(0)

## Field + Bias Sweep

In [ ]:
# initialize the parameteters

# Metadata
sample_name = "X6_L2R4"
date = time.localtime()[1:3]
date = f"{date[0]}-{date[1]}"

# Angle: 82 degrees
bias_current = 2.0e-3 # SHNO bias: 180nm: 2.2mA, 150nm: 2mA
shno_bias.set_output(I=bias_current, V=2, turn_on=False)

# Set bias current range
start_bias = 1.2e-3
stop_bias = 2.5e-3
bias_step = 0.025e-3

total_steps = int((stop_bias-start_bias)/bias_step)
bias_range = np.linspace(start_bias, stop_bias, total_steps)

#applied_field = np.zeros_like(bias_range)


# Set magnet range in output voltage
start_field = 1 #[A]
stop_field = 5  #[A]
step_size = 0.1



magnet_range = np.linspace(start_field, stop_field, int(abs((stop_field-start_field)/step_size)+1))
applied_field = np.zeros_like(magnet_range)


# Angle
#start_angle = 77.2 # set mnually 


#applied_field = np.zeros(angle_steps)

# Turn on FSV
start_f = 3E9; stop_f = 10E9
bandwidth = 2000E3
capture_points = int(2*(stop_f-start_f)/bandwidth)

waveforms = np.zeros((len(magnet_range), capture_points))
waveforms_bg = np.zeros_like(waveforms)
fsv.show_screen(start_f = start_f, stop_f = stop_f, bandwidth=bandwidth, points = capture_points)
frequency = np.linspace(start_f, stop_f, capture_points)


In [ ]:
# Start measurements

lna_psu.lna(state=True)
shno_bias.on()
mag_R = magnet.set_calibrated_current(start_field)
time.sleep(1)


# Start the measurement loop
for bias_current in tqdm(bias_range):

    mag_R = magnet.set_calibrated_current(start_field, mag_R)
    
    progress_bar = tqdm(enumerate(magnet_range), total=len(magnet_range), leave=False)
    for i, field in progress_bar:

        mag_R = magnet.set_calibrated_current(field, mag_R)
        time.sleep(1)
        applied_field[i] = magnet.field_strenght()
        progress_bar.set_postfix({'Applied Field': applied_field[i]})
        waveforms[i] = fsv.get_trace(avg_count=50)
    
    # Save automatically during the measurement for each different bias current. 
    # Create the folder for the data before running the sweep!
    name = f"bias_sweep/field_sweep_{sample_name}/{date}_{sample_name}_" + "{:0.2f}".format((bias_current*1000)) + "_mA_76deg.h5"
    with h5py.File(name, 'w') as f:
        f.create_dataset('frequency', data=frequency)
        f.create_dataset('waveforms', data=waveforms)
        f.create_dataset('applied_field', data=applied_field)
    
    
# Turn off all power
lna_psu.lna(state=False)
shno_bias.off()
magnet.off()
shno_bias.chirp(0)

## Saving and loading data

In [ ]:
# Save the data
name = f'field_sweep/{date}_{sample_name}_{str(bias_current*1000).replace(".", "_")}72_deg_0_5T.h5'
ans = input(f"Confirm saving? [y/n]:{name}")
if ans == 'y':
    with h5py.File(name, 'w') as f:
        # Store the waveforms, frequency, and applied field
        f.create_dataset('frequency', data=frequency)
        f.create_dataset('waveforms', data=waveforms)
        f.create_dataset('applied_field', data=applied_field)
        f.create_dataset('angle', data=angle_range)
        #f.create_dataset('bias_range', data=bias_range)
    print(f"File saved as: '{name}'")
    print("Time:", time.strftime("%H:%M:%S", time.localtime()))
else: 
    print("Save cancelled!")

In [ ]:
# Load data
name = './angle_sweep/field_sweep/X6_L1R2/3-13_2_0mA_80.0deg.h5'
ans = input("Confirm loading? [y/n]")
if ans == 'y':
    with h5py.File(name, 'r') as f:
        applied_field = f['applied_field'][:]
        waveforms = f['waveforms'][:]
        frequency = f['frequency'][:]
    print("Data loaded!")
    print("Time:", time.strftime("%H:%M:%S", time.localtime()))
else: 
    print("Loading cancelled!")
    

## Visualizations

In [ ]:
# Select the type of data that has been captured. 
x_values = applied_field  #bias_range, applied_field, angle_range
y_values = frequency/1e9
y_values = y_values
X, Y = np.meshgrid(x_values, y_values)

plt.pcolormesh(X, Y, (waveforms.T), shading='nearest', cmap='viridis') 
#plt.xlim(1.1, 2.2)
#plt.ylim(3.3, 3.7)
plt.colorbar(label='Amplitude [dBm]')
plt.xlabel('Applied field [mT]')                    # Adjust the x-axis label to the plot type
#plt.xlabel('Out-of-Plane angle [$ \degree $]')
#plt.xlabel('Bias current [mA]')
plt.ylabel('Frequency [GHz]')
plt.tight_layout()
plt.show()

Data can also be vizualized in 3D using different plotting types. 

In [ ]:
# Plotting in 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Loop through the Fourier Transforms and plot them
for i, ft in enumerate(waveforms):
    ax.plot(frequency/1e9, applied_field[i]*1000, zs=ft, zorder=-i,lw=0.5) #label=f"Signal {i+1}"
    
ax.set_xlabel('Frequency [GHz]')
ax.set_zlabel('Amplitude [dBm]')
ax.set_ylabel('Applied field[mT]')
ax.set_title('3D Plot of Captured Spectrum')
plt.tight_layout()

plt.show()

In [ ]:
# Interactive 3D plot

colormap = 'Viridis'
vmin, vmax = -80, -55  # Fixed range in dBm


X, Y = np.meshgrid(frequency / 1e9, applied_field)
Z = np.array(waveforms)

fig = go.Figure(data=[go.Surface(
    x=X, y=Y, z=Z,
    colorscale=colormap,
    cmin=vmin, cmax=vmax,
    colorbar=dict(title="Amplitude [dBm]")
)])

fig.update_layout(
    width=800,
    height=600,
    scene=dict(
        xaxis_title='Frequency [GHz]',
        yaxis_title='Applied field [mT]',
        zaxis_title='Amplitude [dBm]'
    ),
    title='Interactive 3D Plot of Captured Spectrum',
    autosize=True, 
    showlegend=False
)

# Show the plot
fig.show()

## Disconnect instruments 

In [ ]:
vxm.Close()
fsv.close()
lna_psu.close()
magnet.close()